In [1]:
'''
This Jupyter notebook plots altitude profiles of the data of Tromso EISCAT radar.

The file data_2009_2019_TS.mat contains a matlab struct jouleMedians with the following elements:
  jouleMedians.h		  centres of altitude grid points (km)
  jouleMedian.mltbins	  limits of the MLT bins (hours)
  jouleMedians.KPbins	  limits of the KP bins
  jouleMedians.q		  the Joule heating rate profiles (W/m^3)
The array jouleMedians.q is an NKP x NMLT x Nh array:
  - NKP is the number of KP bins (3)
  - NMLT is the number of MLT bins (4)
  - Nh is the number of altitude bins (256). 
Ex: the profile of the 1st KP (KP<=2) bin and the 2nd MLT bin (21-03 MLT) is jouleMedians.q(1,2,:).
The medians are calculated from EISCAT UHF radar data at Tromso, Norway at 67 deg MLAT.

The file EISCAT_JH_percentiles.mat contains a matlab struct joulePercentiles with the following elements:
  joulePercentiles.h	      centres of altitude grid points (km)
  jouleMedian.mltbins	      limits of the MLT bins (hours)
  joulePercentiles.KPbins	  limits of the KP bins
  joulePercentiles.q	      the Joule heating rate profiles (W/m^3)
  joulePercentiles.percentiles  the percentiles (%)
The array joulePercentiles.q is an NKP x NMLT x Nh x N% array:
  - NKP is the number of KP bins (3), 
  - NMLT is the number of MLT bins (4), 
  - Nh is the number of altitude bins (256), 
  - N% is the number of percentiles (5). 
Ex: the median profile of the 1st KP (KP<=2) bin and 2nd MLT bin (21-03 MLT) is joulePercentiles.q(1,2,:,3), 
    and the 10th perentile of the same data is joulePercentiles.q(1,2,:,1).

'''
import math
import numpy as np
import scipy.io

import plotly
import chart_studio.plotly as py 
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

def plotEISCAT( VariableName ):
    '''
    Args:
        VariableName (string): The physical variable on which the calculation has been applied. Valied values: "Joule Heating" or "Pedersen Conductivity"
    '''
    matlabStruct = scipy.io.loadmat('./EISCAT_DATA/data_2009_2019_TS.mat')
    
    allALTs = np.array( matlabStruct[ 'data_2009_2019_TS' ][0][0][0] ).flatten()
    allKPs  = list( np.array( matlabStruct[ 'data_2009_2019_TS' ][0][0][1][0] ) )
    allMLTs = list( np.array( matlabStruct[ 'data_2009_2019_TS' ][0][0][2][0] )[:-1] )
    allJHs  = np.array( matlabStruct[ 'data_2009_2019_TS' ][0][0][3] )
    allPEDs = np.array( matlabStruct[ 'data_2009_2019_TS' ][0][0][4] )
    
    print( "Altitudes:", allALTs[0], allALTs[1], "...", allALTs[-1] )
    print( "KPs:", allKPs  )
    print( "MLTs:", allMLTs )
    print( "JHs shape:", allJHs.shape )
    print( "PEDs shape:", allPEDs.shape )
    print( "" )
    
    if VariableName == "Joule Heating":
        Values = allJHs
        x_axes_range=[0, 20]
        MultiplicationFactor = 10**8 
        new_units = "10^-8 W/m3"
    elif VariableName == "Pedersen Conductivity":
        Values = allPEDs
        x_axes_range=[0, 0.4]
        MultiplicationFactor = 10**3 
        new_units = "mS/m"
    else:
        print( "ERROR: Invalid Variable Name." )
    
    ALTsequence =  allALTs
    MLTsequence = allMLTs
    KPsequence = [ 0, 2, 4 ] 
    MLT_duration_of_a_profile = 6
    
    # alter visibleALTsequence so that the point is displayed in the middle of the sub-bin
    visibleALTsequence = ALTsequence.copy()
    for i in range(1, len(visibleALTsequence)-1):
        visibleALTsequence[i] += 0.5
    
    # plot
    Color10 = '#00FFAA' #'#c4dfe6'
    Color25 = '#00DCAA' #'#a1d6e2'
    Color50 = '#008C78' #'#1995ad'
    Color75 = '#00DCAA' #'#a1d6e2'
    Color90 = '#00FFAA' #'#c4dfe6'
    
    # construct the column MLT titles #("0-3", "3-6", "6-9", "9-12", "12-15", "15-18", "18-21", "21-24")
    ColumnTitles = list()
    
    for i in range(0, len(MLTsequence)):
        s =  "MLT "
        if( MLTsequence[i] > 24 ):
            s += str(MLTsequence[i]-24)
        else:
            s += str(MLTsequence[i])
        s += "-";
        if( MLTsequence[i]+MLT_duration_of_a_profile > 24 ):
            s += str(MLTsequence[i]+MLT_duration_of_a_profile-24) 
        else:
            s += str(MLTsequence[i]+MLT_duration_of_a_profile) 
        ColumnTitles.append( s )
    # define secondary y-axis at the right of the plot
    mySpecs = list()
    for row in range(0, len(KPsequence)):
        mySpecs.append( list() )
        for col in range(0, len(MLTsequence)):
            mySpecs[row].append( {"secondary_y": True} )

    #make plot
    if VariableName == "Joule Heating": 
        XXtitle = 'Joule heating (10<sup>-8</sup> W/m<sup>3</sup>)'
    elif VariableName == "Pedersen Conductivity":
        XXtitle = 'Pedersen conductivity (mS/m)'
    fig = make_subplots(rows=len(KPsequence), cols=len(MLTsequence), x_title=XXtitle, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.035, horizontal_spacing=0.02, subplot_titles=ColumnTitles, specs=mySpecs)
    
    # set fonts
    fig.update_layout( font=dict( family="arial black", size=22 ) )
    fig.update_annotations( font=dict( family="arial black", size=24 ) )
    #fig.update_xaxes(title_font_family="Arial black", title_font_size=20)
    #fig.update_yaxes(title_font_family="Arial black", title_font_size=20)
    fig.update_xaxes(tickfont_size=22)
    fig.update_yaxes(tickfont_size=22)
    fig.layout.annotations[4]["font"] = {'size': 30}  # this is the XXtitle at the bottom
    
    
    for aKP in KPsequence:
        for aMLT in MLTsequence:
            #Means = list()
            Percentiles10 = list()
            Percentiles25 = list()
            Percentiles50 = list()            
            Percentiles75 = list()
            Percentiles90 = list()
            hits  = 0
            
            # compute percentiles
            Percentiles10 = Values[KPsequence.index(aKP), MLTsequence.index(aMLT), :, 0] * MultiplicationFactor
            Percentiles25 = Values[KPsequence.index(aKP), MLTsequence.index(aMLT), :, 1] * MultiplicationFactor
            Percentiles50 = Values[KPsequence.index(aKP), MLTsequence.index(aMLT), :, 2] * MultiplicationFactor #Percentiles50 = JHmedians[1,1,:] * MultiplicationFactor 
            Percentiles75 = Values[KPsequence.index(aKP), MLTsequence.index(aMLT), :, 3] * MultiplicationFactor
            Percentiles90 = Values[KPsequence.index(aKP), MLTsequence.index(aMLT), :, 4] * MultiplicationFactor
            
            fig.add_trace( go.Scatter(x=[0]*len(visibleALTsequence), y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color10, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles10, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color10, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles25, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color25, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles50, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color50, line=dict(color='black',width=2,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            # plot mean
            #fig.add_trace( go.Scatter(x=Means, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor='black', line=dict(color='black',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            # plot percentiles
            fig.add_trace( go.Scatter(x=Percentiles75, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color75, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
            fig.add_trace( go.Scatter(x=Percentiles90, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color90, line=dict(color='gray',width=1,), showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1,  )
            # add a trace in order to display secondary y-axis at the right
            fig.add_trace( go.Scatter(x=[-1000], y=[-1000], showlegend=False), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1, secondary_y=True )
            
            
            # compute maximums of median
            PanelMax = 0
            Alt_of_Max = 0
            for i in range (0,len(ALTsequence)):
                if PanelMax < Percentiles50[i] * MultiplicationFactor:
                    PanelMax = Percentiles50[i] * MultiplicationFactor
                    Alt_of_Max = ALTsequence[i]
            print( "MAXIMUM at", aKP, aMLT, ":", PanelMax, "at", Alt_of_Max, "km" )
            #fig.add_annotation(xref='x domain', yref='y domain', x=0.97, y=0.90, text=F"max at <b>{int(Alt_of_Max)}-{int(Alt_of_Max+1)}km</b>", showarrow=False, row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1, font=dict(color="red") )
            
            
            # CALCULATE area under median curve
            area = 0.0
            for i in range(0, len(Percentiles50)):
                if math.isnan(Percentiles50[i]) == False: 
                    if VariableName == "Joule Heating": 
                        area += Percentiles50[i]*0.01; #area += Percentiles50[i]*1000 * math.pow(10,-8) * 1000;
                    else: 
                        area += Percentiles50[i];
            if VariableName == "Joule Heating":
                print ( aKP, aMLT, "Height Integration of Median:", round(area,2), "mW/m2")
                fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=0, text=F"<b>{round(area,2)}&nbsp;mW/m<sup>2</sup></b>", showarrow=False, row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1, font=dict(color=Color50) )
            else:
                print ( aKP, aMLT, "Height Integration of Median:", round(area,2), "S")
                fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=0, text=F"<b>{round(area,2)}&nbsp;S</b>", showarrow=False, row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1, font=dict(color=Color50) )
            
    # display legends
    '''
    fig.add_trace( go.Scatter(name='10th Perc.', x=Percentiles10, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color10, line=dict(color='gray',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    fig.add_trace( go.Scatter(name='25th Perc.', x=Percentiles25, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color25, line=dict(color='gray',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    fig.add_trace( go.Scatter(name='50th Perc.', x=Percentiles50, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color50, line=dict(color='black',width=2,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    #fig.add_trace( go.Scatter(name='Mean value', x=Means, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor='#5cc5ef', line=dict(color='black',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )            
    fig.add_trace( go.Scatter(name='75th Perc.', x=Percentiles75, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color75, line=dict(color='gray',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    fig.add_trace( go.Scatter(name='90th Perc.', x=Percentiles90, y=visibleALTsequence, mode='lines', fill='tonexty', fillcolor=Color90, line=dict(color='gray',width=1,), showlegend=True), row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1 )
    '''

    fig.update_xaxes( range=x_axes_range, row=1, col=1)
    fig.update_xaxes( range=x_axes_range, row=1, col=2)
    fig.update_xaxes( range=x_axes_range, row=1, col=3)
    fig.update_xaxes( range=x_axes_range, row=1, col=4)
    
    fig.update_xaxes( range=x_axes_range, row=2, col=1)
    fig.update_xaxes( range=x_axes_range, row=2, col=2)
    fig.update_xaxes( range=x_axes_range, row=2, col=3)
    fig.update_xaxes( range=x_axes_range, row=2, col=4)
    
    fig.update_xaxes( range=x_axes_range, row=3, col=1)
    fig.update_xaxes( range=x_axes_range, row=3, col=2)
    fig.update_xaxes( range=x_axes_range, row=3, col=3)
    fig.update_xaxes( range=x_axes_range, row=3, col=4)
    
    
    for aKP in KPsequence:
        fig.update_yaxes( title_text="Altitude(km)", row=KPsequence.index(aKP)+1, col=1, side='left', secondary_y=False)
        row_title = "Kp " + str(aKP) + " - "
        if aKP == 0:
            row_title +=  "2"
        elif aKP == 2:
            row_title +=  "4"
        else:
            row_title +=  "9"
        fig.update_yaxes( title_text=row_title, row=KPsequence.index(aKP)+1, col=len(MLTsequence),  side='right', secondary_y=True, showticklabels=False )
        for aMLT in MLTsequence:
            fig.update_yaxes( row=KPsequence.index(aKP)+1, col=MLTsequence.index(aMLT)+1, secondary_y=True, showticklabels=False )
    #fig.update_xaxes( range=x_axes_range )
    fig.update_yaxes( range=[80, 150], tick0=90, dtick=20 )  
    # title = "EISCAT" + "  " + "Alt.Prof. of " + VariableName + " (" + new_units + ")"
    fig.update_layout( title='EISCAT (2009-2019)' , title_font_color=Color50,
                       width=400+len(MLTsequence)*250, height=200+200*len(KPsequence), showlegend=True, legend_orientation="h", legend_y=-0.04) 

    
    # Add annotations to each subplot regarding the number of samples 
    if VariableName=="Joule Heating":
        # number of meassurements
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={4912}", showarrow=False, row=1, col=1)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={3030}", showarrow=False, row=1, col=2)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={1964}</b>" , showarrow=False, row=1, col=3)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={5658}", showarrow=False, row=1, col=4)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={2320}", showarrow=False, row=2, col=1)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={1662}</b>" , showarrow=False, row=2, col=2)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={1346}</b>" , showarrow=False, row=2, col=3)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={2470}</b>" , showarrow=False, row=2, col=4)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={196}</b>"  , showarrow=False, row=3, col=1)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={158}</b>"  , showarrow=False, row=3, col=2)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={150}</b>"  , showarrow=False, row=3, col=3)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={72}</b>"  , showarrow=False, row=3, col=4)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
    else:
        # number of meassurements
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={7118}", showarrow=False, row=1, col=1)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={4258}", showarrow=False, row=1, col=2)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={3150}</b>" , showarrow=False, row=1, col=3)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={6020}", showarrow=False, row=1, col=4)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={2804}", showarrow=False, row=2, col=1)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={1886}</b>" , showarrow=False, row=2, col=2)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={1442}</b>" , showarrow=False, row=2, col=3)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={2530}</b>" , showarrow=False, row=2, col=4)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={210}</b>"  , showarrow=False, row=3, col=1)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={164}</b>"  , showarrow=False, row=3, col=2)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={150}</b>"  , showarrow=False, row=3, col=3)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        fig.add_annotation(xref='x domain', yref='y domain', x=0.99, y=1.0, text=F"N={74}</b>"  , showarrow=False, row=3, col=4)
        fig.layout.annotations[-1]["font"] = {'family':'arial', 'size':18}
        
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig ) 

    # plot more zoom versions
    '''
    new_x_axes_range = [x * (2/3) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    new_x_axes_range = [x * (1/2) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    new_x_axes_range = [x * (3/2) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    new_x_axes_range = [x * (2.5) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    new_x_axes_range = [x * (10) for x in x_axes_range]
    fig.update_xaxes( range=new_x_axes_range )
    plotly.offline.iplot(fig) 
    '''
    
plotEISCAT("Joule Heating")
plotEISCAT("Pedersen Conductivity")